This notebook develops an agent-based model to simulate the growth of a train network. The notebook has 5 segements: 
1- Importing the necessary packages
2- Reading the inputs layer and preprocess them based on the contex of Sydney data
3- Defining the interaction functions 
4- Runing the model
5- Visualization

## 1- Imports

In [1]:
### Import packages
import numpy as np
import pandas as pd
import geopandas as gpd
# from scipy.spatial import distance
import networkx as nx
# import momepy # a package for urban morphological analysis
import time, enum, math
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
# import osmnx as ox
# import folium
from pyproj import CRS
import warnings; warnings.simplefilter('ignore')

## 2- Inputs and preprocessing

In [2]:
### Reading inputs
Centroids_df = pd.read_csv('./input/MB_Centroids.csv') #Reading MB centroids 
Access_df = pd.read_csv('./input/WalkAccess.csv') #Reading walk access for centriods
Access_Job_df = pd.read_csv('./input/WalkJobAccess_allYears.csv') #Reading walk access for centriods
Current_stations = gpd.read_file('./input/TrainStations.shp')
Suburbs = gpd.read_file('./input/Suburbs.shp')
Current_train_lines = gpd.read_file('./input/Train_lines.shp')
# Exclusions = gpd.read_file('./input/Exclusions.shp') #Reading the exclusion areas based on the planning requirements
# Exclusions['exclusion'] = True
# Exclusions = pd.read_csv('./input/MB_categories.csv')
# Exclusions = Exclusions[(Exclusions.MB_CAT16 == 'Parkland') | (Exclusions.MB_CAT16 == 'Water')].MB_CODE16.values

### filtering 15min walk access to pop from the database
Access_pop = Access_df[['MB_CODE16']]
for year in [i for i in range(1851,2017)]:
    Access_pop = Access_pop.join(Access_df[str(year)+'_15min_walk'])
for col in Access_pop.columns[1:]:
    name = col.split('_15min_walk')[0]
    Access_pop = Access_pop.rename(columns={col:name})
    
### filtering 15min walk access to jobs from the database
Access_job = Access_Job_df[['MB_CODE16']]
for year in [i for i in range(1851,2017)]:
    Access_job = Access_job.join(Access_Job_df[str(year)+'_15min_walk'])
for col in Access_job.columns[1:]:
    name = col.split('_15min_walk')[0]
    Access_job = Access_job.rename(columns={col:name})

Access = Access_Job_df[['MB_CODE16']]
for year in [i for i in range(1851,2017)]:
    Access[str(year)] = round((2 * Access_job[str(year)] + Access_pop[str(year)])/3,1)
    
### edit and prepare inputs
Centroids = Centroids_df.copy()
# Centroids= Centroids[Centroids.MB_CODE16.isin(Exclusions)== False]
# Centroids.reset_index(inplace=True)
Centroids['HasStation'] = False   #MBs that have access to at least one station in 1km walk direct distance ~15min walk
Centroids = gpd.GeoDataFrame(Centroids,crs='EPSG:4326',geometry = gpd.points_from_xy(Centroids['lon'],Centroids['lat']))
# Centroids = Centroids.to_crs(3112)

### filtering Cetroids based on the exclusion factors
# Centroids = gdp.sjoin(Centroids, Exclusions,op='intersects',how='inner')
# Centroids = Centroids[Centroids.exclusion != True]

Suburbs= gpd.GeoDataFrame(Suburbs[['SSC_CODE_2','geometry']])
Suburbs = Suburbs.to_crs(4326)
MB_layer = Centroids.merge(Access,on='MB_CODE16')
Suburbs_df = gpd.sjoin(Suburbs, MB_layer ,op='contains',how='inner') #join train station layer into into boundary shapes and summarize it by number of stops at each boundary
Suburbs_df = Suburbs_df.groupby('SSC_CODE_2').agg('mean')
Suburbs_df.reset_index(inplace=True)
Stations = gpd.sjoin(Current_stations , Suburbs ,op='intersects',how='inner')
Stations['Active'] = False

## 3- Defining the model

In [ ]:
"""Defining all the necessary functions"""

##############################################################################################################################################

def azimuth(point1, point2):
    '''azimuth between 2 shapely points (interval 0 - 360)'''
    angle = np.arctan2(point2.x - point1.x, point2.y - point1.y)
    return np.degrees(angle) if angle >= 0 else np.degrees(angle) + 360

##############################################################################################################################################

def optimum_point(point,time,angle):
    '''finding the optimum location in the second search space'''
    # define goldilock zone
    buffer = point.buffer(D*0.00001).difference(point.buffer(d*0.00001)) # *0.00001 will convert meter to degree
    #possible points on goldilock zone
    points = gpd.clip(Centroids[Centroids.HasStation == False],buffer)
    points = points.merge(Access[['MB_CODE16',str(time)]],on='MB_CODE16')
    #it can be the center of the best cluster
    points['azimuth'] = points['geometry'].map(lambda x: azimuth(point,x))
    points['angle'] = points['azimuth'].map( lambda x: abs(x - angle) if abs(x - angle) <= 180 else (360-abs(x - angle)))
    points['angle_cost'] = points['angle'].map(lambda x: math.ceil(x/theta)**2) #new method
    points['distance'] = points['geometry'].map(lambda x: point.distance(x)*111) #111 converts distance to km
    points['link_cost'] = points['angle_cost'] * points['distance']
    points['BC_ratio'] = points[str(time)] / points['link_cost']
    return points['BC_ratio'].argmax()

##############################################################################################################################################

def next_stop_2nd_order(point,time,angle):
    '''search in the goldilocks zone to find the best MB for the next station'''
    try:
        # define goldilock zone
        buffer = point.buffer(D*0.00001).difference(point.buffer(d*0.00001)) # *0.00001 will convert meter to degree
        #possible points on goldilock zone
        points = gpd.clip(Centroids[Centroids.HasStation == False],buffer)
        points = points.merge(Access[['MB_CODE16',str(time)]],on='MB_CODE16')
        points['2nd_order_ratio'] = points.apply(lambda x: optimum_point(x['geometry'],time,angle),axis = 1)
        points['azimuth'] = points['geometry'].map(lambda x: azimuth(point.iloc[0].geometry,x))
        points['angle'] = points['azimuth'].map( lambda x: abs(x - angle) if abs(x - angle) <= 180 else (360-abs(x - angle)))
        points['angle_cost'] = points['angle'].map(lambda x: math.ceil(x/theta)**2) #new method
        points['distance'] = points['geometry'].map(lambda x: point.iloc[0].geometry.distance(x)*111) #111 converts distance to km
        points['link_cost'] = points['angle_cost'] * points['distance']
        points['2nd_BC_ratio'] = points[str(time)] / points['link_cost']
        points['weights'] = points['2nd_order_ratio'] + points['2nd_BC_ratio']
        second_best_point_ratio = points['weights'].nlargest(2).values[1] #pick the second highest BCR
        best_option = points.loc[[points['weights'].argmax()]].drop(columns=[str(time),'2nd_order_ratio','distance','link_cost','2nd_BC_ratio']) #'weights'
        best_option['2nd_weight'] = second_best_point_ratio
        best_option['2nd_best_point'] = points.loc[points['weights'].nlargest(2).index[1]].geometry
        return best_option
    except:
        return None


##############################################################################################################################################

def Activate(point):
    '''this makes the centroids around a station to be deactivated in the search space for next runs. HasStation= access to stations'''
#     Stations.at[point.stop_id.values[0],'Active']=True
    buffer = point.buffer(station_buffer*0.00001)
    polygon= buffer.values[0] # *0.00001 will convert meter to degree
    Centroids.loc[Centroids.within(polygon),'HasStation']=True

##############################################################################################################################################
    
def initiate():
    '''set up the necessary varablies with initial conditions. This only need to be run only once.'''
    global G, Start_point, Centroids, intitial_azimuth, System_ticks, initial_points, \
            D, d, station_buffer, theta , L, N, branching_factor, Lines, Terminals, Directions, End, \
            Transfer_Gap, max_circuity, Conncetion_Angle, Running_system
    
    
    Centroids['HasStation'] = False   #MBs that have access to at least one station in 1km walk direct distance ~15min walk
    #MB_CODE16 of some stations
    initial_points = {'Central':10757290000, 'Redfern':10752184000, 'Granville':10596561000, 'Penrith':10615372000}    
    D = 1500 # The bigger buffer radius in meter
    d = 1000 # The bigger buffer radius in meter
    station_buffer = 1000 # the buffer around stations representing 15min walk access
    theta = 30 #setting the angle for limiting the variation (based on alpha:the degree between 2 lines)
    intitial_azimuth = 220 #the initial direction of development
    Start_point = Centroids[Centroids.MB_CODE16 == initial_points['Central']] 
        
    Lines = {}     # dictionary of lines: keys are line names, and the values are the line_df includes of stations
    Terminals = {} # dictionary of end points: keys are the line names and the values are the list of each line terminus
    Directions ={} # Dictionary of last segment's direction (azimuth) of each line: keys are the name of liness and values are the azimuths of each line 
    End = 0        #End is 0 or 1: 0 is towards initial direction, 1 is backward
#     Transfer_Gap = 1 #Gaps between transfer stations
    G = nx.Graph() # empty graph
    L = 10          # Maximum number of lines in the system
    N = 50          # Number of stations development per year
#     branching_factor = 1 # a factor for incrasing the cost of branching
#     max_circuity = 1.5  # Maximum circuity to Central
    Conncetion_Angle = 50 #The alowable connection angle for reconnection rule. It is similar to theta angle
    System_ticks = 1
    Running_system = True

##############################################################################################################################################

def Branch(line,point):
    '''Define a new line at a transfer station/point'''
    global Lines,Terminals
    if len(Lines) < L:
        key = 'Line_L' + str(len(Lines)+1)
        if key not in Lines:
            Lines[key] = pd.DataFrame()
            Lines[key] = Lines[key].append(point)
            Lines[key]['stop_seq'] = 1
            Lines[key]['Transfer'] = True
            Terminals[key+ '_End0'] = pd.DataFrame()
            Terminals[key+ '_End1'] = pd.DataFrame()
            Terminals[key+ '_End0'] = Terminals[key+ '_End0'].append(point) #End0 and End1 are two end points of a line
            Terminals[key+ '_End1'] = Terminals[key+ '_End1'].append(point)
            if len(Lines) == 1:
                Lines[key]['Transfer'] = False
                Lines[key]['azimuth'] = intitial_azimuth
                Directions[key+ '_End0'] = intitial_azimuth
                Directions[key+ '_End1'] = Directions[key+ '_End0']+180 if Directions[key+ '_End0']<180 else Directions[key+ '_End0']-180
                Terminals[key+ '_End0']['azimuth'] = Directions[key+ '_End0']
                Terminals[key+ '_End1']['azimuth'] = Directions[key+ '_End1']
            else:
                #making the transfer station of the last line as True
                Lines[line].reset_index(drop=True,inplace=True)
                position = Lines[line].loc[Lines[line].MB_CODE16 == point.iloc[0].MB_CODE16].index[0]
                Lines[line].at[position,'Transfer'] = True
                #measuring the direction of the next line
                Directions[key+ '_End0'] = azimuth(Lines[key].iloc[-1].geometry,Lines[key].iloc[-1]['2nd_best_point'])
                Directions[key+ '_End1'] = Directions[key+ '_End0']+180 if Directions[key+ '_End0']<180 else Directions[key+ '_End0']-180
        else:
            pass
    else:
        pass
    
##############################################################################################################################################

def Next_staion():
    '''define the growth pattern'''
    global Lines, Terminals,Running_system,End, Start_point
    if len(Lines) == 0:
        Start_point['Transfer'] = False
        Branch('Line_L1',Start_point)
        Activate(Start_point)
    else:
        pass
    
    #Checking connection rules
    NextConnections = {k: Should_connect(k.split('_End')[0],v) for k,v in Terminals.items() if v is not None}
    Connections= {k:v[1] for k,v in NextConnections.items() if v[0]}
    if len(Connections) !=0:
        for k,v in Connections.items():
            WhichLineEnd = k
            NextStation = v
            WhichLine = WhichLineEnd.split('_End')[0]
            End = int(WhichLineEnd.split('_End')[1])
            ToReconnect(WhichLine,NextStation,End)
    else:    
        #find the next possible moves for all the lines at their terminals
        NextOptions = {k: next_stop_2nd_order(v,year,Directions[k]) for k,v in Terminals.items()}
        #filter None values
        NextOptions = {k:v for k,v in NextOptions.items() if v is not None}
        #find the best line that gives the highest BCR 
        if len(NextOptions)!=0:
            #for next edits you should change try, because it does not return back error if any
            WhichLineEnd = max(NextOptions.items(), key= lambda x: x[1]['weights'].values)[0] #0 gives the key 1 gives the value
            NextStation = NextOptions[WhichLineEnd]
            WhichLine = WhichLineEnd.split('_End')[0]
            End = int(WhichLineEnd.split('_End')[1])

            Should_branch(WhichLine,NextStation)
            Add_station(WhichLine,NextStation,End)
            circuity_check(WhichLine,End)
        else:
            #if no other options left
            Running_system = False
        
##############################################################################################################################################

#old version
# def Should_branch(line,station):
#     '''check whether a line should do a branch when wants to add a new station'''
#     #check if the next station is better than the second best station of previous station
#     if len(Lines[line]) > 1:
#         if (branching_factor * station.iloc[0]['weights']) < Lines[line].iloc[-1]['2nd_weight']:
#             return True
#         else:
#             return False
#     else:
#         return False
    
def Should_branch(line,station):
    '''check whether a line should do a branch when wants to add a new station'''
    #check if the next station is better than the second best station of previous station
    if len(Lines[line]) > 1:
        non_transfers = Lines[line][Lines[line].Transfer == False]
        transfers = Lines[line][Lines[line].Transfer == True]
        transfers_positions = transfers.stop_seq.tolist()
        neighbours= []
        for i in transfers_positions:
            neighbours.extend(range(i-Transfer_Gap,i+Transfer_Gap))
        neighbours = list(set(neighbours))
        selectable_stations = non_transfers[~non_transfers.stop_seq.isin(neighbours)]        
        try: 
            if ((branching_factor * station.iloc[0]['weights']) < selectable_stations.iloc[selectable_stations['2nd_weight'].argmax()]['2nd_weight']):
                #make the next transfer point's Transfer = True
                selectable_stations['Transfer'] = True
                Branch(line, selectable_stations.iloc[[selectable_stations['2nd_weight'].argmax()]])
                
            else:
                return
        except:
            return
    else:
        return
    
##############################################################################################################################################

def Should_connect(line,station):
    '''check whether a line should connect to the existing station in a fixed radius buffer'''
    #check if the next station is better than the second best station of previous station
    if len(Lines) > 1:
        if station.iloc[0].Transfer:
            return (False,None)
        else:
            other_lines = Lines.copy()
            #remove the current line from the search
            del other_lines[line]
            options = pd.DataFrame()
            for k,v in other_lines.items():
                non_transfers = v[v.Transfer == False]
                transfers = v[v.Transfer == True]
                transfers_positions = transfers.stop_seq.tolist()
                neighbours= []
                for i in transfers_positions:
                    neighbours.extend(range(i-Transfer_Gap,i+Transfer_Gap))
                neighbours = list(set(neighbours))
                selectable_stations = non_transfers[~non_transfers.stop_seq.isin(neighbours)]
                selectable_stations['FromWhichLine']=k
                options = options.append(selectable_stations)

            buffer = station.buffer(D*0.00001) # *0.00001 will convert meter to degree
            #possible points on the station buffer
            points = gpd.clip(options,buffer)

            if len(points) == 0:
                return (False,None)
            else:
                angle = station.iloc[0]['azimuth']
                points['azimuth1'] = points['geometry'].map(lambda x: azimuth(station.iloc[0].geometry,x))
                points['angle1'] = points['azimuth1'].map( lambda x: abs(x - angle) if abs(x - angle) <= 180 else (360-abs(x - angle)))
                points = points[points['angle1']<= Conncetion_Angle]
                if len(points) > 0:
                    connection_point = points.iloc[[0]].drop(columns=['azimuth1','angle1'])
                    return (True,connection_point)
                else:
                    return (False,None)
    else:
        return (False,None)

##############################################################################################################################################

def circuity(point1,point2,Graph):
    '''calculate circuity between two points'''
    network_distance = nx.shortest_path_length(Graph, source=point1.iloc[0].MB_CODE16, target=point2.iloc[0].MB_CODE16, weight='distance', method='dijkstra')
    euclidean_distance = point1.iloc[0].geometry.distance(point2.iloc[0].geometry)*111
    try:
        C = network_distance/euclidean_distance
    except ZeroDivisionError:
        C = 0
    return C

##############################################################################################################################################

def circuity_check(line,end):
    global Lines, Terminals
    '''check if the circuity of a line exceeds the thresholds'''
    if end==0:
        if circuity(Start_point,Lines[line].sort_values(by=['stop_seq']).iloc[[-1]],G) > max_circuity:
            Terminals[line+'_End0'] = None
    else:
        if circuity(Start_point,Lines[line].sort_values(by=['stop_seq']).iloc[[0]],G) > max_circuity:
            Terminals[line+'_End1'] = None

##############################################################################################################################################

def Add_station(line,station,end):
    '''adding the next station to its assigned line'''
    global G,Directions,Terminals,Lines
    if end ==0:
        #go forward
        last_seq = Lines[line].stop_seq.max()
        station['stop_seq'] = last_seq + 1
    else:
        #go backward
        last_seq = Lines[line].stop_seq.min()
        station['stop_seq'] = last_seq - 1
    station['Transfer'] = False
    station['time'] = year
    Lines[line] = Lines[line].append(station)
    Activate(station)
    Last_direction = azimuth(Lines[line].iloc[-2].geometry,Lines[line].iloc[-1].geometry)
    if end == 0:
        Directions[line+'_End0'] = Last_direction
        Terminals[line+'_End0'] = station
        last_station = Lines[line].sort_values(by=['stop_seq']).iloc[[-2]]
    else:
        Directions[line+'_End1'] = Last_direction
        Terminals[line+'_End1'] = station
        last_station = Lines[line].sort_values(by=['stop_seq']).iloc[[1]]
    
    G.add_edge(last_station.iloc[0].MB_CODE16, station.iloc[0].MB_CODE16, distance = last_station.iloc[0].geometry.distance(station.iloc[0].geometry)*111, line_name = line, time=year)


##############################################################################################################################################

def ToReconnect(line,station,end):
    '''conneting the end point if the conncetion is true'''
    global G,Directions,Terminals,Lines
    if end ==0:
        #go forward
        last_seq = Lines[line].stop_seq.max()
        station['stop_seq'] = last_seq + 1
    else:
        #go backward
        last_seq = Lines[line].stop_seq.min()
        station['stop_seq'] = last_seq - 1
    station['Transfer'] = True
    station['time'] = year
    
    #making the transfer station of the last line as True
    From_which_line = station.iloc[0].FromWhichLine
    Lines[From_which_line].reset_index(drop=True,inplace=True)
    position = Lines[From_which_line].loc[Lines[From_which_line].MB_CODE16 == station.iloc[0].MB_CODE16].index[0]
    Lines[From_which_line].at[position,'Transfer'] = True
    
    station.drop(columns=['FromWhichLine'],inplace=True)
    
    Lines[line] = Lines[line].append(station)
    Last_direction = azimuth(Lines[line].iloc[-2].geometry,Lines[line].iloc[-1].geometry)
    if end == 0:
        Directions[line+'_End0'] = Last_direction
        Terminals[line+'_End0'] = station
        last_station = Lines[line].sort_values(by=['stop_seq']).iloc[[-2]]
    else:
        Directions[line+'_End1'] = Last_direction
        Terminals[line+'_End1'] = station
        last_station = Lines[line].sort_values(by=['stop_seq']).iloc[[1]]
    
    G.add_edge(last_station.iloc[0].MB_CODE16, station.iloc[0].MB_CODE16, distance = last_station.iloc[0].geometry.distance(station.iloc[0].geometry)*111, line_name = line, time=year)

##############################################################################################################################################

def step(n=1):
    '''do the following functions in one or n steps'''
    for x in range(n):
        if Running_system:
            Next_staion()
#              evaluate() #evaluate if a branch can happen or not (you can also compare the number of stations per line)
#              evaluate based on BCR
            tick()
        else:
            break

##############################################################################################################################################
def tick():
    '''increse the system tick by one step'''
    global System_ticks
    System_ticks += 1


## 5.1- Running in a loop

In [ ]:
# B_list=[1] #branching_factor values
# C_list=[1.9,1.7,1.5,1.3] #max_circuity values
# G_list=[1,2,3,4,5,6,7,8,9,10] #Transfer_Gap

B_list=[1] #branching_factor values
C_list=[1.3] #max_circuity values
G_list=[10] #Transfer_Gap

# years = [i for i in range(1851,2016,5)]
years = [i for i in range(1851,2016)]

for b in B_list: 
    branching_factor = b
    for c in C_list:
        max_circuity = c
        for g in G_list:
            Transfer_Gap = g
            
            initiate()
            start_time = time.time()
            for year in years:
                step()
            print('running time: {seconds} seconds'.format(seconds=(time.time()-start_time)))
            ############################################
            Lines['Line_L1'].reset_index(drop=True,inplace=True)
            position = Lines['Line_L1'].loc[Lines['Line_L1'].stop_seq == 1].index[0]
            Lines['Line_L1'].at[position,'time']=1851
            
            
            #save graph
            run_id = 'BF'+ str(b) + '_C' + str(int(c*10)) + '_G' + str(g)
            graph_path = "./output_08/" + run_id + ".edgelist"
            nx.write_edgelist(G, path= graph_path)
            #save shapes
            
            #Saving shp file
            for k,v in Lines.items():
                v['line'] = k
            train_lines_df = pd.concat([v.sort_values(by=['stop_seq']) for k,v in Lines.items()])
            train_lines_df.drop(columns=['2nd_best_point'],inplace=True)
            train_lines_df= gpd.GeoDataFrame(train_lines_df)
            train_lines = gpd.GeoDataFrame()

            #saving stations
            station_path = './output_08/' + run_id + '_Stations.shp'
            train_lines_df.to_file(station_path)

            keys = list(Lines.keys())
            for key in keys:
                df = train_lines_df[train_lines_df.line == key]
                list_n = df.stop_seq.tolist()
                time_n = df.time.tolist()
                for t in time_n:
                    try:
                        train_line_n =  gpd.GeoDataFrame(index=[0],crs='epsg:4326', geometry=[LineString(df[(df.time <= t)].sort_values(by=['stop_seq']).geometry.tolist())])
                        train_line_n['step'] = t
                        train_line_n['line'] = key
                        train_line_n['year'] = t
                        train_lines = train_lines.append(train_line_n)
                    except:
                        continue
            #saving lines
            lines_path = './output_08/' + run_id + '_Lines.shp'
            train_lines.to_file(lines_path)
            print('{file_name_} results saved'.format(file_name_= run_id ))

## 6- Visualization 

In [ ]:
train_lines_df = pd.concat([v for k,v in Lines.items()])
train_lines_df.drop(columns=['2nd_best_point'],inplace=True)
fig, ax = plt.subplots(figsize=(15,8))

train_lines = {k: gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[LineString(v.sort_values(by=['stop_seq']).geometry.tolist())]) for k,v in Lines.items() if len(v) > 1}
train_lines_df = pd.concat([v for k,v in Lines.items()])
train_lines_df.drop(columns=['2nd_best_point'],inplace=True)

colors={'Line_L1':'green' ,'Line_L2':'yellow' , 'Line_L3':'orange','Line_L4':'pink', 'Line_L5':'blue','Line_L6':'cyan','Line_L7':'magenta','Line_L8':'darkorange','Line_L9':'violet','Line_L10':'olive',
       'Line_L11':'green' ,'Line_L12':'yellow' , 'Line_L13':'orange','Line_L14':'pink', 'Line_L15':'blue','Line_L16':'cyan','Line_L17':'magenta','Line_L18':'darkorange','Line_L19':'violet','Line_L20':'olive'}
for k,v in train_lines.items():
    color_ = colors[k]
    v.plot(ax=ax, color = color_)

Suburbs.plot(ax=ax, facecolor = 'grey',alpha=0.8)
train_lines_df.plot(ax=ax, color = 'red', markersize = 5 )
Current_train_lines.plot(ax=ax, color='blue',alpha=0.2 ) 
ax.set_axis_off()
ax.set_xlim([151,151.3])
ax.set_ylim([-34,-33.8])
fig